# Portfolio Optimiser

### Portfolio Optimization using Modern Portfolio Theory (MPT) in Python.

Modern Portfolio Theory (MPT), developed by Harry Markowitz, is a fundamental concept in investment management. It provides a framework for investors to build portfolios that optimize returns for a given level of risk tolerance.

The core principles behind MPT are as follows:

### Risk-Return Trade-off:  
    MPT acknowledges the inherent relationship between risk and return in investments. Higher potential returns are typically associated with higher risks of loss.

### Diversification:  
    A core principle of MPT is diversification. By holding a variety of assets with different risk-return profiles in your portfolio, you can potentially reduce the overall risk without sacrificing expected returns.

### Expected Return and Risk:  
    MPT focuses on expected return, the average return an investor anticipates from an investment over a specific period ad well as quantified risk measurement (volatility/standard deviation)

### Efficient Frontier:  
    MPT helps identify the set of optimal portfolios that offer the highest expected return for a given level of risk, or the lowest risk for a given expected return. This set of portfolios is visualized as the "efficient frontier."

### Risk Aversion:  
    MPT assumes investors are generally risk-averse. Given two portfolios with similar expected returns, investors would prefer the one with lower risk.



In [217]:
# ----------- Importing Modules ----------- #
import numpy as np
import pandas as pd
import seaborn as sn
import time
from datetime import datetime
from bokeh.models import ColumnDataSource, CrosshairTool, HoverTool, NumeralTickFormatter
from bokeh.plotting import figure, show

### User defined global variables, classes, and functions

### Variables

In [218]:
''' 
    Initialise all variables, note that the risk free rate of return is 
    usally defined by the rate of return on government bonds
'''

api_key = "Your API Key"
trade_days = 252
risk_free_return = 0
random_seed = 255
num_iter = 1000

### Classes

In [219]:
# Initialise a portfolio class

''' 
    _portfolio  = dataframe storing portfolio data
    _stock_list = list of stocks in the portfolio
    _weight     = weight of each stock, stored as vector
    _rfr        = risk free rate of return
'''
class _Portfolio:
    # defined risk free rate of return, default set at 2%, adjust according to country specific rfr
    rfr = 0.01

    # initialise class with portfolio dataframe, weight vector, and risk-free-return rate properties
    def __init__(self, _portfolio, _stock_list, _weight):
        self.port       = _portfolio
        self.stock_list = _stock_list
        self.weight     = _weight

    # set the risk free rate of return
    def set_rfr(self, rate):
        rfr = rate

    # calculates returns portfolio % change from previous column    
    def returns_port(self):
        return self.port.pct_change().ffill()
    
    # calculates the covariance matrix, multiply by number of trade days
    def cov_matrix(self):
        return self.returns_port().cov()
    
    # calculation of porfolio variance, np.multidot() can't be used here since matrix multiplication isn't commutative
    def port_var(self):
        return np.dot(np.transpose(self.weight),np.dot(self.cov_matrix(),self.weight))

    # calcualtion of porfolio volatility
    def port_vol(self):
        return np.sqrt(self.port_var())
    
    # annualise the porfolio volatility
    def port_vol_annualised(self):
        return np.sqrt(self.port_var())*np.sqrt(trade_days)    

    # Monte Carlo method of generating randomised portfolios
    def montecarlo_portfolio_generator(self, port_iter = 1000):
        # initialise lists for portfolio returns, volatility, and weights
        port_returns = []
        port_volatility = []
        port_weights = []

        # calculate average inidividual returns on each stock/asset
        individual_ret = self.returns_port().mean()
        num_assets = len(self.port.columns)
        
        # set random seed
        np.random.seed(random_seed)

        for p in range(port_iter):
            # randomly generate weights, such that they sum up to 1
            # append to the portfolio weights list
            weights = np.random.random(num_assets)
            weights = weights/np.sum(weights)
            self.weight = weights
            port_weights.append(weights)

            # returns = weights dot product individual E[returns]
            # append to returns list
            returns = np.dot(individual_ret, weights)
            port_returns.append(returns)

            # compute portfolio variance, standard deviation, and volatility
            vol = self.port_vol_annualised()
            port_volatility.append(vol)

        # create a dictionary of returns & volatility
        dict_ret_vol = {'returns':port_returns, 'volatility':port_volatility}
        
        for i, stock in enumerate(self.port.columns.tolist()):
            dict_ret_vol[stock] = [w[i] for w in port_weights]
        
        _mc = pd.DataFrame(dict_ret_vol)
        _mc = _mc.dropna()
        _mc = _mc.reset_index(drop=True)

        return _mc
      
    # returns the minimum volatility portfolio
    def min_vol_portfolio(self):
        port_sim = self.montecarlo_portfolio_generator()
        return port_sim.loc[port_sim['volatility'].idxmin()]
    
    # returns the optimal sharpe ratio portfolio
    def sharpe_ratio_portfolio(self):
        port_sim = self.montecarlo_portfolio_generator()
        p = port_sim
        p['sharpe'] = (port_sim['returns']-self.rfr)/port_sim['volatility']
        return port_sim.loc[p['sharpe'].idxmax()]
    
    def max_return_portfolio(self):
        port_sim = self.montecarlo_portfolio_generator()
        return port_sim.loc[port_sim['returns'].idxmax()]

### Global Functions

In [220]:
# Import dependencies, yahoo finance, requests, and session rate limiters
import yfinance as yf
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter
from pandas_datareader import data as pdr

class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND*5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)

session.headers['User-agent'] = 'Portfolio_Optimiser ver 1.0'

# override with pandas_datareader data format
yf.pdr_override()

# define a function to generate a dataframe from data obtained from google finance
def generate_portfolio(stock_list, start_date, end_date):
    data = pdr.get_data_yahoo(stock_list,start=start_date,end=end_date,session=session)
    return data

### Main function

The historical data is obtained via the Alpha Vantage API, for more information visit 
https://www.alphavantage.co/documentation/. The user will need to register to obtain a presonal API key. (non-commercial use of the API is free)

In [221]:
'''
Define all the variable inputs here
'''
# Define start/end dates here, as well as the personal API key obtained
start_date = '2019-01-01'
end_date = '2024-01-01'
# api_key = input("Please input your finnhub API key:")

# read CSV file for list of stocks
csv_file = input("Please input file path:")
try:
    stocks_file = pd.read_csv(csv_file, delimiter=None, header=None)
    stock_list = stocks_file.iloc[:,0].tolist()
except FileNotFoundError:
    print("Error: File not found!")

### Portfolio generation and calculations

In [222]:
# Generate a porfolio, accessing the data from the API
portfolio_data = generate_portfolio(stock_list, start_date, end_date)

[*********************100%%**********************]  6 of 6 completed


In [223]:
# Preformat the dataframe before declaring the class object
portfolio_data = portfolio_data.iloc[:,:len(stock_list)]
portfolio_data = portfolio_data.set_axis(stock_list, axis='columns')

# Generate a random initial weighting
num_stocks = len(stock_list)
w = np.random.random(num_stocks)
w = w/np.sum(w)

In [224]:
# Initialise an instance of the Portfolio class
portfolio_v1 = _Portfolio(portfolio_data, stock_list, w)

In [225]:
# find the minimum volatility, max sharpe ratio, max returns portfolio
min_volatiliy_portfolio_v1 = portfolio_v1.min_vol_portfolio()

print(min_volatiliy_portfolio_v1)

returns       0.001439
volatility    0.304025
AAPL          0.228374
GOOGL         0.265785
AMZN          0.059946
NVDA          0.331716
CELH          0.077794
TSLA          0.036385
Name: 710, dtype: float64


In [226]:
max_return_portfolio_v1 = portfolio_v1.max_return_portfolio()

print(max_return_portfolio_v1)

returns       0.003070
volatility    0.481900
AAPL          0.030895
GOOGL         0.082567
AMZN          0.439226
NVDA          0.075235
CELH          0.121701
TSLA          0.250376
Name: 6, dtype: float64


In [227]:
sharpe_portfolio_v1 = portfolio_v1.sharpe_ratio_portfolio()

print(sharpe_portfolio_v1)


returns       0.003037
volatility    0.492672
AAPL          0.088537
GOOGL         0.148493
AMZN          0.519777
NVDA          0.044322
CELH          0.090647
TSLA          0.108224
sharpe       -0.034431
Name: 895, dtype: float64


In [228]:
'''
Plots the efficiency frontier of the various simulated portfolios, 
and plots pie charts for the lowest risk portfolio, the optimal sharpe ratio portfolio, and the maximal returns portfolio
'''
portfolio_mc = portfolio_v1.montecarlo_portfolio_generator()

p = figure(height=700, width=800, title="Efficient frontier. Number of simulated portfolios: " + str(num_iter),
            tools='box_zoom,wheel_zoom,reset', toolbar_location='above')
p.add_tools(CrosshairTool(line_alpha=1, line_color='black', line_width=1))
p.add_tools(HoverTool(tooltips=None))
data_source = {'risk': portfolio_mc['volatility'], 'returns': portfolio_mc['returns']}
source = ColumnDataSource(data=data_source)
p.scatter(x='risk', y='returns', source=source, line_alpha=0, hover_color='navy', alpha=0.4, hover_alpha=1, size=8)
show(p)


In [229]:
min_vol = portfolio_v1.min_vol_portfolio()
max_sharpe = portfolio_v1.sharpe_ratio_portfolio()
max_return = portfolio_v1.max_return_portfolio()
p.scatter(min_vol['returns'], min_vol['volatility'], color='blue', legend_label='Portfolio with minimum risk', size=10)
p.scatter(max_sharpe['returns'], max_sharpe['volatility'], color='green', legend_label='Portfolio with maximal Sharpe ratio', size=12)
p.scatter(max_return['returns'], max_return['volatility'], color='red', legend_label='Portfolio with maximal return', size=9)
p.legend.location = "top_left"
p.xaxis.axis_label = 'Volatility/Risk (standard deviation)'
p.yaxis.axis_label = 'Annual return'
p.xaxis[0].formatter = NumeralTickFormatter(format="0.0%")
p.yaxis[0].formatter = NumeralTickFormatter(format="0.0%")
show(p)

### Assumptions of MPT

Market Efficiency:
 MPT assumes a relatively efficient market where all available information is reflected in asset prices. Prices adjust quickly to new information, and there are no arbitrage opportunities.

Normal Distribution of Returns:
 MPT often uses statistical measures like standard deviation to quantify risk. This assumes asset returns are normally distributed.

Stationary Returns and Correlations:
 MPT calculations rely on historical data to estimate expected returns and correlations between assets. The theory assumes these characteristics are relatively stable over time.

Focus on Mean-Variance Optimization:
 MPT optimizes portfolios based on mean (expected return) and variance (risk) but doesn't consider other potential return distributions or risk measures.

### Limitations of MPT

The assumptions differ from what the market is in reality, and are the factors that limit the application of MPT, in the real world,
the market isn't perfectly efficient, the returns aren't necessarily normally distributed, and the returns/volatility interpolated from historical data doesn't necessarily correlate to behaviour in the future. Thus the model itself can be supplemented with machine learning algorithms, and consideration of more variables in terms of risks and returns.